In [1]:
import music21

In [8]:
import random




In [8]:
%%writefile midi_preproc.py

from music21 import converter, stream, note, chord, duration, pitch
from scipy.sparse import csc_matrix

import numpy as np

instruments = ['Piano']

def midiToMatrix(filename):
    parsed = converter.parse(filename)

    party = []

    for part in parsed:
        for voice in part.getElementsByClass(stream.Voice):
            if voice.getInstrument().instrumentName in instruments:
                for thisNote in [n for n in voice if (isinstance(n,note.Note) or isinstance(n,chord.Chord))]:
                    cur_chord = np.zeros(129)
                    for _pitch in thisNote.pitches:
                        cur_chord[_pitch.midi] = 1
                        #text += pitch.name+str(pitch.octave)
                    dur = thisNote.duration.quarterLength
                    cur_chord[-1] = dur#dur.numerator / float(dur.denominator)

                    party.append(cur_chord)
                    #text += dur_to_text(thisNote.duration.type)+'z'

    return csc_matrix(party)

from music21 import midi

def save_mat2_mid(sparsed, fname='output/test.mid'):
    music_stream = stream.Stream()

    for dense_line in np.array(sparsed.todense()):
        (notes,) = np.where(dense_line[:-1]>0.5)

        pitches = []
        for n in notes:
            pitches.append(pitch.Pitch(midi=n))

        crd = chord.Chord(notes= pitches, quarterLength=dense_line[-1])
        music_stream.append(crd)
        
    md = midi.translate.streamToMidiFile(music_stream)
    md.open(fname, 'wb')
    md.write()
    md.close()

Writing midi_preproc.py


In [6]:
#test
sparsed = midiToMatrix('midi/abide_.mid')

In [7]:
save_mat2_mid(sparsed)

In [5]:
%%writefile dataset_generation.py

SEQ_LEN = 200
STRIDE_MU = 20 #expected value of stride
STRIDE_SIG = 2 #standard standard deviation of stride

import sys
from os import listdir
from random import choice
from midi_preproc import midiToMatrix
import numpy as np
import scipy
from prefetch_generator import background

bad_files = []

#all files in storage
files = [each for each in listdir('midi') if each.endswith('.mid') and (not ('midi/'+each in bad_files))]

def split_random_file():#returns sparse matrix every SEQ_LEN rows of wich are cut from the vectorized midi file
    fi = choice(files)#select random midi file
    
    cut = []
    
    sparsed = midiToMatrix('midi/'+fi)
    pos = 0
    
    #cut the sequence
    while(sparsed.shape[0] - pos > SEQ_LEN):
        
        cut.append(sparsed[pos: pos+SEQ_LEN])
        
        pos+= np.clip(np.round(np.random.normal(STRIDE_MU, STRIDE_SIG)), 1, SEQ_LEN//2)
    if len(cut) > 0:
        return scipy.sparse.vstack(cut,format='csr')
    else:
        return scipy.sparse.csc_matrix(np.zeros(shape=(0,129)))
    
def generate_minibatch(batch_size=32):
    res = []
    while(sum([x.shape[0] for x in res])/SEQ_LEN < batch_size):
        res.append(split_random_file())
    return scipy.sparse.vstack(res,format='csr')

@background(max_prefetch=10)
def iterate_minibatches(num_batches, batch_size=32):
    for i in xrange(num_batches):
        yield generate_minibatch(batch_size)

Overwriting dataset_generation.py


In [2]:
from dataset_generation import generate_minibatch

In [31]:
test_rnd_cutter = split_random_file()

In [3]:
test_generate = generate_minibatch()

ress


In [4]:
test_generate

<8200x129 sparse matrix of type '<type 'numpy.float64'>'
	with 25189 stored elements in Compressed Sparse Row format>